In [4]:
import aiohttp
import pprint
import json
pp = pprint.PrettyPrinter(indent=4)
headers={"Authorization": "Basic cm9vdDpyb290"}
base_url='http://guillotina:8081/db'
async with aiohttp.ClientSession(headers=headers) as sess:
    async with sess.post(
            base_url,
            data=json.dumps({
                "@type": "Container",
                "id": "cms"
            })) as resp:
        pp.pprint(await resp.json())
    async with sess.post(
            base_url + '/cms/@addons',
            data=json.dumps({
                "id": "cms"
            })) as resp:
        pp.pprint(await resp.json())
    async with sess.post(
            base_url + '/cms/@addons',
            data=json.dumps({
                "id": "dbusers"
            })) as resp:
        pp.pprint(await resp.json())

{'@type': 'Container', 'id': 'cms', 'title': 'cms'}
{   'available': [   {   'dependencies': [],
                         'id': 'cms',
                         'title': 'Guillotina CMS'},
                     {   'dependencies': [],
                         'id': 'dbusers',
                         'title': 'Guillotina DB Users'}],
    'installed': ['cms']}
{   'available': [   {   'dependencies': [],
                         'id': 'cms',
                         'title': 'Guillotina CMS'},
                     {   'dependencies': [],
                         'id': 'dbusers',
                         'title': 'Guillotina DB Users'}],
    'installed': ['dbusers', 'cms']}


In [5]:
import csv
import sys
import uuid
csvfile = open('/app/raw_data/Articles.csv', encoding='iso8859-1')
articles = csv.reader(csvfile, delimiter=',')

# Pass heading
next(articles)
total_articles = 2579
good = 0
conflict = 0
errors = 0

async with aiohttp.ClientSession(headers=headers) as sess:
    async with sess.post(
            base_url + '/cms',
            data=json.dumps({
                "@type": "Folder",
                "id": "articles"
            })) as resp:
        if resp.status == 201:
            pp.pprint(await resp.json())
    for article in articles:
        uid = uuid.uuid4().hex
        async with sess.post(
                base_url + '/cms/articles',
                data=json.dumps({
                    "@type": "Document",
                    "title": article[2],
                    "text": {
                        "content_type": "text",
                        "encoding": "iso8859-1",
                        "data": article[0]
                    },
                    "guillotina_cms.interfaces.tiles.ITiles": {
                        "tiles": {
                            uid + "title": {"@type": "title"},
                            uid + "body": {"@type": "text"}
                        },
                        "tiles_layout": {
                            "items": [
                                uid + "title",
                                uid + "body"
                            ]
                        }
                    },
                    "guillotina.behaviors.dublincore.IDublinCore": {
                        'effective_data': article[1],
                        'tags': [article[3]]
                    }
                })) as resp:
            if resp.status == 201:
                good += 1
            elif resp.status == 409:
                conflict += 1
            else:
                errors += 1
            print('{} good, {} conflicts, {} bad - {} / {} complete.'.format(
                good, conflict, errors, good + conflict + errors, total_articles), end='\r')
            sys.stdout.flush()

csvfile.close()

{   '@id': 'http://guillotina:8081/db/cms/articles',
    '@name': 'articles',
    '@type': 'Folder',
    '@uid': 'e4f|6330c41e857a45cfaffc61a8fd6824bd',
    'UID': 'e4f|6330c41e857a45cfaffc61a8fd6824bd',
    'title': 'articles'}
